In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
# from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
import matplotlib
# be able to save images on server
matplotlib.use('Agg')
from matplotlib import pyplot
print(tf.__version__)


2.10.0


In [ ]:
# Zip
# importing the "tarfile" module
import tarfile # type: ignore

# open file
file = tarfile.open('1_150x9.tar.gz')

# extracting a specific file
file.extractall(path='./1_150x9/')

file.close()

In [ ]:
# Data generator
def data():
    timestepsPerSample = 150

    # Replace 'your_file.csv' with the actual file path
    file_features = './1_150x9/1_150x9f.csv'
    file_labels = './1_150x9/1_150x9l.csv'

    # Specify the CSV file name
    csv_file_name = 'my_data1.csv'

    # Read the .csv file and create an array
    data_strings = np.genfromtxt(file_features, delimiter=';')
    labels_strings = np.genfromtxt(file_labels,delimiter=';')

    # Wycinanie wybranych kolumn
    # data_s = data_strings[:,[0,2,3,4,5,6,7,8]]
    data_s = data_strings[:,[0,4,5,8]]
    num_features = data_s.shape[1]
    print(data_strings[:3])
    print(data_s[:3])
    
    # Convert from strings to float and int
    X = data_s.astype(float).reshape((-1,timestepsPerSample,num_features))
    Y = labels_strings.astype(float).reshape((-1,3))
    print(X.shape)
    print(Y.shape)

    # Modification from imported to new size
    X_mod = X[1000:11000,130:]
    Y_mod = Y[1000:11000]
    timestepsPerSample = X_mod.shape[1]
    print(X_mod.shape)
    print(Y_mod.shape)
    print(X_mod[:1])

    # Splitting
    x_train, x_test, Y_train, Y_test = train_test_split(X_mod,Y_mod, test_size=0.15, shuffle=False)
    print(x_train.shape)
    y_train = Y_train[:, 0:2]
    y_test = Y_test[:, 0:2]
    print(y_train[:2])
    print(y_test[:2])
    return x_train, x_test, y_train, y_test, Y_test

In [2]:
# fit lstm model
def fit_lstmModel(x_train, y_train, batch_size, nb_epoch, neurons):
    LstmLayer = LSTM(
    units=neurons,
    activation="tanh",
    recurrent_activation="sigmoid",
    use_bias=True,
    kernel_initializer="glorot_uniform",
    recurrent_initializer="orthogonal",
    bias_initializer="zeros",
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.2,
    recurrent_dropout=0.0,
    seed=None,
    return_sequences=False,
    return_state=False,
    go_backwards=False,
    stateful=False,
    unroll=False,
    input_shape=(x_train.shape[1],x_train.shape[2])
    )
    model = Sequential() # initializing model
    # input layer and LSTM layer with 50 neurons
    model.add(LstmLayer)
    model.add(Dense(100, activation='relu'))
    # model.add(Dense(100, activation='relu'))
    model.add(Dense(20, activation='relu'))
    # outpute layer with sigmoid activation
    model.add(Dense(y_train.shape[1], activation='sigmoid'))

    num_samples = x_train.shape[0]
    STEPS_PER_EPOCH = num_samples/batch_size
    # lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.001,
    decay_steps=STEPS_PER_EPOCH*100,
    decay_rate=0.9,
    staircase=False)

    def get_optimizer():
         # return tf.keras.optimizers.Adam(learning_rate=0.001)
        return tf.keras.optimizers.Adam(learning_rate= lr_schedule) # type: ignore

    optimizer = get_optimizer()

    # defining loss function, optimizer, metrics and then compiling model
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

    # fit model
    model.fit(x_train, y_train, epochs=nb_epoch, batch_size=batch_size, shuffle=False, verbose=0) # type: ignore
    return model

In [ ]:
# Profit
def funcProfit(predict, Y_test):
    predict_classes = np.where(predict > 0.5, 1,0)
    concat = np.hstack((predict_classes, Y_test[:,2:]))
    df = pd.DataFrame(concat, columns=['Sell', 'Buy', 'Price'])

    sum = 0
    for i in range(0, len(df)):
        if (i-1) > 0:
            if df.at[i,'Sell'] == 1:
                sum += (df.at[i,'Price'] - df.at[i-1,'Price'])*(-1)
            elif df.at[i,'Buy'] == 1:
                sum += (df.at[i,'Price'] - df.at[i-1,'Price'])
            if df.at[i,'Sell'] != df.at[i-1,'Sell']:
                sum -= 0.03
    return sum

In [3]:
# Experiment
def experiment(repeats, epochs):
    # Data gen
    x_train, x_test, y_train, y_test, Y_test = data()

    # Evaluate declaration
    accuracy = list()
    profit = list()

    # Repeats
    for r in range(repeats):
        batch_size = 4
        # model
        model = fit_lstmModel(x_train, y_train, batch_size, epochs, 5)

        # forecast the entire training dataset to build up state for forecasting
        predict = model.predict(x_test, batch_size=batch_size)
        test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0) # type: ignore

        accuracy.append(test_acc*100)
        profit.append(funcProfit(predict, Y_test))
    
    return accuracy, profit

In [4]:
# experiment
repeats = 30
results_acc = DataFrame()
results_profit = DataFrame()
# vary training epochs
epochs = [500, 1000, 2000, 4000, 6000]
for e in epochs:
 results_acc[str(e)], results_profit[str(e)] = experiment(repeats, e)
# summarize results
print(results_acc.describe())
# save boxplot
results_acc.boxplot()
pyplot.savefig('boxplot_epochs.png')






        500 1000 2000 4000 6000
count     0    0    0    0    0
unique    0    0    0    0    0
top     NaN  NaN  NaN  NaN  NaN
freq    NaN  NaN  NaN  NaN  NaN


ValueError: boxplot method requires numerical columns, nothing to plot.